In [ ]:
import requests

url = "https://www.iiserbpr.ac.in/people/faculty"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers, verify=False)  # disable SSL verification
html = response.text

print(html[:500])


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iiserbpr.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta name="google-site-verification" content="JIn0JQlmj5Z1k5AUpZnb6AQbee1qRQSFiEKN2YfrPhg" />
  <title>People</title>
  <link rel="stylesheet" href="https://www.iiserbpr.ac.in/assets/css/bootstrap.min.css">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css" />

  <link rel="stylesheet" href="


In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://www.iiserbpr.ac.in"
FACULTY_PAGE = f"{BASE_URL}/people/faculty"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(FACULTY_PAGE, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Select all faculty cards
faculty_cards = soup.select("div.col-sm-6.col-md-4.col-lg-4.mt-4.mb-4")

with open("iiserbpr_faculty.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Name", "Designation", "Email", "Image URL", "Profile Link", "PhD", "Postdoc"])

    for card in faculty_cards:
        # Name
        name_tag = card.select_one("div.offcheading")
        name = name_tag.text.strip() if name_tag else ""

        # Designation
        desig_tag = card.select_one("ul.contact-info li.name span.text")
        designation = desig_tag.text.strip() if desig_tag else ""

        # Email
        email_tag = card.select_one("ul.contact-info li a[href^='mailto:']")
        email = email_tag.text.strip() if email_tag else ""

        # Image
        img_tag = card.select_one("img.officerimg")
        img_url = img_tag.get("src").strip() if img_tag else ""

        # Profile Link
        profile_tag = card.select_one("a.btn[href^='https']")
        profile_link = profile_tag.get("href").strip() if profile_tag else ""

        # Initialize PhD and Postdoc
        phd = ""
        postdoc = ""

        # Fetch profile page if link exists
        if profile_link:
            try:
                profile_resp = requests.get(profile_link, headers=headers)
                profile_soup = BeautifulSoup(profile_resp.text, "html.parser")

                # Example: Look for text like "Ph.D" or "Postdoctoral" in h6 or p tags
                for h6_tag in profile_soup.find_all("h6"):
                    text = h6_tag.get_text(strip=True)
                    if "Ph.D" in text or "PhD" in text:
                        phd = text
                    elif "Postdoctoral" in text or "Postdoc" in text:
                        postdoc = text

                # Small delay to be polite to the server
                time.sleep(0.5)
            except Exception as e:
                print(f"Failed to fetch {profile_link}: {e}")

        # Write row to CSV
        writer.writerow([name, designation, email, img_url, profile_link, phd, postdoc])

print("Scraping completed! Data saved to iiserbpr_faculty.csv")


SSLError: HTTPSConnectionPool(host='www.iiserbpr.ac.in', port=443): Max retries exceeded with url: /people/faculty (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))